In [1]:
# %matplotlib widget
# %matplotlib inline
%matplotlib qt5

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from qick import *
from qick.helpers import gauss
from tqdm import tqdm_notebook as tqdm

import time
import os
import sys
sys.path.append('/home/xilinx/jupyter_notebooks/')
import scipy as sp
import json
from scipy.fft import fft, fftfreq
import Pyro4.util

from slab.instruments import *
from slab.experiment import Experiment
from slab.datamanagement import SlabFile
from slab import get_next_filename, AttrDict

# Figure params
plt.rcParams['figure.figsize'] = [10,6]
plt.rcParams.update({'font.size': 14})

# Load experiments

In [ ]:
# %aimport experiments
# NOTE: ADDING NEW METHODS WILL NOT BE UPDATED
import experiments as meas

In [8]:
"""Reopen saved data"""
def prev_data(expt_path, filename=None):
    temp_data_file = expt_path
    if filename is not None: temp_data_file = os.path.join(expt_path, filename)
    print(temp_data_file)
    with SlabFile(temp_data_file) as a:
        attrs = dict()
        for key in list(a.attrs):
            attrs.update({key:json.loads(a.attrs[key])})
        keys = list(a)
        temp_data = dict()
        for key in keys:
            temp_data.update({key:np.array(a[key])})
        print(f'Extracted data and attrs {list(a.attrs)}')
    return temp_data, attrs

In [ ]:
# expt_path=os.path.join(os.getcwd(), 'data', 'data_240617')
data_path = 'S:\\QRAM\\qram_4QR2\\'
expt_path = os.path.join(data_path, 'data', 'data_250119')
print('Data will be stored in', expt_path)

In [ ]:
# config_file = 'config_zcu216.yml'
# config_file = 'config_q3diamond.yml'
# config_file = 'config_q3diamond_full688_reset.yml'
config_file = 'config_q3diamond_full688and638_reset.yml'
# config_path = os.path.join(os.getcwd(), config_file)

# config_file = 'config.yml'
config_path = os.path.join('s:\\Connie\\experiments\\qram_tprocv1_expts\\configs\\', config_file)

print('Config will be', config_path)

# Setup experiment hardware

Set in config:
- LO alias, frequency, power
- SOC dac channels, nyquist zones

In [ ]:
import yaml
print('Using config file', config_path)
with open(config_path, 'r') as cfg_file:
    yaml_cfg = yaml.safe_load(cfg_file)
yaml_cfg = AttrDict(yaml_cfg)

In [ ]:
# im = InstrumentManager(ns_address='192.168.14.1') # Chicago lab
# im = InstrumentManager(ns_address='192.168.137.1') # SLAC lab
# im = InstrumentManager(ns_address='10.108.30.56') # Spilker Fermium
im = InstrumentManager(ns_address='10.108.30.75') # Spilker Vibranium
# print(im)
print(im.keys())

In [ ]:
soc = QickConfig(im[yaml_cfg['aliases']['soc']].get_cfg())
print(soc)
# print(soc.description())
# print(soc.dump_cfg())

In [ ]:
lo1 = im[yaml_cfg.aliases.readout_LO]
# lo1 = SignalCore(name="SignalCore", address='10003494')
lo1.open_device()
print(lo1.get_id())

# TURN OFF
lo1.set_standby(True)
lo1.set_output_state(False)

# print(lo1.get_device_status().pll_status.crs_pll_ld)

# # TURN ON
# lo1.set_standby(False)
# lo1.set_output_state(True)
# # lo_freq = float(yaml_cfg.hw.lo.readout.frequency)
# # lo_power = float(yaml_cfg.hw.lo.readout.power)
# lo_freq = 4385.5e6
# lo_power = 14
# lo1.set_frequency(lo_freq) # Set LO frequency
# print(f'set lo freq to {lo_freq*1e-6} MHz')
# lo1.set_power(lo_power) # Set LO power
# print(f'set lo power to {lo_power} dBm')

# Single qubit RB

In [11]:
qubit_i = 2
# qubits = [2, 1]
# qubit_i = qubits[0]

In [ ]:
rbprog = meas.SimultaneousRBExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"rb1Q_qubit{qubit_i}",
    config_file=config_path,
)

gate_char = None
# gate_char = 'X'

# max_gates = 225

max_gates = 100
# expts = 30
expts = 5

if gate_char is None: step = max_gates // expts
else: step = max_gates // (2*expts)
print('step', step)


rbprog.cfg.expt = dict(
    # start=1, # rb depth start
    start=34, # rb depth start
    step=step, # step rb depth
    # step=5, # step rb depth
    # expts=expts, # number steps
    expts=1, # number steps
    reps=5000, # number averages per unique sequence
    variations=3, # number different sequences per depth
    gate_char=None, # single qubit clifford gate (str) to characterize. if not None, runs interleaved RB instead of regular RB
    qubits=[qubit_i], # the qubits to perform simultaneous RB on (if eg-gf, q should be qA != 1)
    # qubits=qubits,
    # qDrive=qubits[0],
    # singleshot_reps=20000, # reps per state for singleshot calibration
    singleshot_reps=100, # reps per state for singleshot calibration
    post_process='threshold', # 'threshold' (uses single shot binning), 'scale' (scale by ge_avgs), or None

    # thresholds=[0, 613.9114184246247, 507.47891189954305, 0],
    # angles=[0, -101.54512016437148, 72.02480620589141, 0],
    # ge_avgs=[array([0., 0., 0., 0.]), [-101.85060937209302, 466.51286116279067, -154.72153951162792, 742.5887439767441], [21.863250678241997, -256.37994772110693, 120.73579574064026, -794.2915984536081], array([0., 0., 0., 0.])],
    # counts_calib=[[16238, 3110, 527, 125], [3374, 16034, 95, 497], [1968, 395, 14344, 3293], [494, 2404, 2838, 14264]]
)
# rbprog.cfg.device.qubit.pulses.pi_ge.gain[qubit_i] = int(0.8*rbprog.cfg.device.qubit.pulses.pi_ge.gain[qubit_i])
# rbprog.cfg.device.readout.relax_delay = 1
# rbprog.cfg.device.readout.readout_length = 5

print(rbprog.cfg)

import Pyro4.util
try:
    rbprog.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

# rbprog.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
rbprog.analyze(fit=True)
rbprog.display(fit=True, qubit=[qubit_i])

In [ ]:
p_irb = 0.913
p_rb = 0.949  
print(100*meas.fitting.rb_error(p_rb, 2),'% average error')
print(100*(1-meas.fitting.rb_gate_fidelity(p_rb, p_irb, 2)), '% gate error')

In [ ]:
qDrive = qubits[0]
setup_ZZ = 0

In [ ]:
rbprog.save_data()

### EF RB

In [22]:
qubit_i = 3

ZZ_qubit = 0 # for initialization
# ZZ_qubit = None

test_qZZ = ZZ_qubit

In [ ]:
rbprog = meas.SimultaneousRBEFExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"rb1Q_qubit{qubit_i}",
    config_file=config_path,
)

# gate_char = None
gate_char = 'X'

max_gates = 225
# max_gates = 100
# max_gates = 150
# max_gates = 40

expts = 30
# expts = 5

if gate_char is None: step = max_gates // expts
else: step = max_gates // (2*expts)
print('step', step)

rbprog.cfg.expt = dict(
    start=1, # rb depth start
    # start=200, # rb depth start
    step=step, # step rb depth
    # expts=1, # number steps
    expts=expts+1, # number steps
    reps=2000, # number averages per unique sequence
    reps_f=5000,
    # reps=200, # number averages per unique sequence
    # reps_f=500,
    loops=1,
    variations=15, # number different sequences per depth
    gate_char=gate_char, # single qubit clifford gate (str) to characterize. if not None, runs interleaved RB instead of regular RB
    qubits=[qubit_i], # the qubits to perform simultaneous RB on (if eg-gf, q should be qA != 1)
    ZZ_qubit=ZZ_qubit,
    test_qZZ=test_qZZ,
    # qubits=qubits,
    # qDrive=qubits[0],
    singleshot_reps=20000, # reps per state for singleshot calibration
    # singleshot_reps=100, # reps per state for singleshot calibration
    post_process='threshold', # 'threshold' (uses single shot binning), 'scale' (scale by ge_avgs), or None
    measure_f=[qubit_i],

)

# print(rbprog.cfg)

import Pyro4.util
try:
    rbprog.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

# rbprog.go(analyze=False, display=False, progress=True, save=False)

In [ ]:
rbprog.save_data()

In [ ]:
rbprog.analyze(fit=True, separate_correction=True)
rbprog.display(fit=True, show_all_vars=False)

# SWAP RB

## Calibrate swap phase

In [28]:
swap_qubit = 2

# ZZ_qubit = 0 # for tomography pulses
ZZ_qubit = None

test_pi_half = False

use_IQ_pulse = False
use_robust_pulses = True
full_mux_expt = True
resonator_reset = [0, 2, 3]

In [ ]:
phaseprog = meas.EgGfPhaseExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"EgGfPhaseCalib{swap_qubit}",
    config_file=config_path,
)

n_pulses = 8
# n_pulses = 2
if test_pi_half: n_pulses *= 2

qubit = 1

start = 0
span = 180
npts = 10

print('step size', span/npts)

phaseprog.cfg.expt = dict(
    start_phase=start,
    step_phase=span/npts,
    expts_phase=npts+1,
    reps=10000,
    singleshot_reps=10000,
    # singleshot_reps=100,
    swap_qubit=swap_qubit,
    test_pi_half=test_pi_half,
    n_pulses=n_pulses,
    qubit=qubit,
    use_IQ_pulse=use_IQ_pulse,
    use_robust_pulses=use_robust_pulses,
    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)
for q in range(4):
    if q != qubit:
        phaseprog.cfg.device.readout.gain[q] = 1e-4

# print(phaseprog.cfg)

import Pyro4.util
try:
    phaseprog.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
phaseprog.save_data()

In [ ]:
if swap_qubit == 2:
    target_X_counts = [1, 0]
elif swap_qubit == 3:
    target_X_counts = [0.78406, 0.21594]
else:
    assert False

phaseprog.analyze(target_X_counts=target_X_counts)
phaseprog.display()

## Calibrate phase between X/2 and X

In [24]:
qubits = [2, 1]
qDrive = 2

# ZZ_qubit = 0 # for tomography pulses
ZZ_qubit = None

test_pi_half = False

use_IQ_pulse = False
use_robust_pulses = True
full_mux_expt = True
resonator_reset = [0, 2, 3]

In [ ]:
phaseprog = meas.EgGfX_Xhalf_PhaseCalib_Experiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"EgGfX_Xhalf_PhaseCalib{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

n_cycles = 16

qubit = 1

start = 0
span = 180
npts = 15

print('step size', span/npts)

phaseprog.cfg.expt = dict(
    start=start,
    step=span/npts,
    expts=npts+1,
    reps=5000,
    loops=1,
    singleshot_reps=10000,
    # singleshot_reps=100,
    n_cycles=n_cycles,

    post_process=None,
    measure_f=False,
    qDrive=qDrive,
    qubits=qubits,

    use_IQ_pulse=use_IQ_pulse,
    use_robust_pulses=use_robust_pulses,
    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
)
for q in range(4):
    if q not in qubits:
        phaseprog.cfg.device.readout.gain[q] = 1e-4

# print(phaseprog.cfg)

import Pyro4.util
try:
    phaseprog.go(analyze=False, display=False, progress=True, save=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

In [ ]:
phaseprog.display(fit=False)

## Run RB

### Test RB

In [128]:
# qubits = [2, 1]
# qDrive = 2

qubits = [3, 1]
qDrive = 3

full_mux_expt = False
resonator_reset = [0, 2, 3] if full_mux_expt else None
use_robust_pulses = True

In [129]:
step_sb = 15
end_sb = 100

step_leak = 15
end_leak = 300
sb = np.arange(1, end_sb+step_sb, step_sb)
lk = np.arange(sb[-1]+step_leak, end_leak+step_leak, step_leak)
depths = np.concatenate((sb, lk))


# depths = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200, 250, 300, 350, 400]
depths = [1]

print('depths', len(depths))
print(depths)

depths 1
[1]


In [130]:
rbprog = meas.SimultaneousRBEgGfExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"rb_EgGf_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

gate_char = None
# gate_char = 'X'


# # expts = 30
# # expts = 19
# expts = 10

# # if gate_char is None: max_gates = 500
# if gate_char is None: max_gates = 100
# else: max_gates = 300

# step = max_gates // expts
# step = 10
# print('step', step)

rbprog.cfg.expt = dict(
    depths=depths,

    # variations=20,
    variations=1,

    # reps=3000, # number averages per unique sequence need at least 10,000 for a good amount of error per point!!
    # reps_f=3000,
    reps=10000, # number averages per unique sequence need at least 10,000 for a good amount of error per point!!
    # reps_f=1000000,
    # reps=100,
    # loops=10,
    loops=1,
    gate_char=gate_char, # single qubit clifford gate (str) to characterize. if not None, runs interleaved RB instead of regular RB
    qubits=qubits,
    qDrive=qDrive,
    singleshot_reps=10000, # reps per state for singleshot calibration
    # singleshot_reps_f=15000, # reps per state for singleshot calibration
    # singleshot_reps=800, # reps per state for singleshot calibration
    post_process='threshold', # 'threshold' (uses single shot binning), 'scale' (scale by ge_avgs), or None

    measure_f_only=False,
    measure_f=[qDrive],
    add_phase=True,

    # cool_qubits=[0, 1],

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    use_robust_pulses=True,

    validate_variations=False,

)
# rbprog.cfg.device.qubit.pulses.pi_ge.gain[qubit_i] = int(0.8*rbprog.cfg.device.qubit.pulses.pi_ge.gain[qubit_i])
# rbprog.cfg.device.readout.relax_delay = 1
# rbprog.cfg.device.readout.readout_length = 5

print(rbprog.cfg)

import Pyro4.util
try:
    rbprog.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

# rbprog.go(analyze=False, display=False, progress=True, save=False)

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-389.975, 5795.761666663333, 502.368233333334, -228.9], 'gain': [0.5, 0.6, 0.4, 0.3], 'readout_length': [5, 20, 10, 8], 'frequency_ef': [-391.91408405, 5795.76166666, 500.44439227, -230.30732892], 'gain_ef': [0.05, 0.6, 0.4, 0.25], 'readout_length_ef': [1, 20, 10, 8], 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4111.144451095202, 4105.783958808882, 4111.0028964412795, 4110.5966963096635, 3442.9795552267146, 3448.3315918098933, 3447.233490336077, 3447.0285717924894, 4762.767009306428, 4761.793655204115, 4762.902435440665, 4762.367822139689, 4380.662337410765, 4379.890920412824, 4380.6668186549305, 4381.242475074487], 'f_ef': [3885.3117884920853, 0.0, 0.0, 0.0, 0.0, 3347.8480888529375, 0.0, 0.0, 4571.9056482340775, 4571.070566218902, 4572.9717022650675, 0.0, 4211.5805515224565, 4207.169953136575, 0.0, 4209.889752285018], 'f_EgGf': [2686.53594683, 0, 2017.3, 2414.92850999], 'f_EgGf_Q_half':

  0%|          | 0/6 [00:00<?, ?it/s]

Qubit (1) ge
ge fidelity (%): 63.45843284328432 	 angle (deg): 14.4423397293289 	 threshold ge: 71.68120421057847
Qubit (3) ge
ge fidelity (%): 77.32 	 angle (deg): 13.1106455942161 	 threshold ge: 54.47102126354997
thresholds=[0, 71.68120421057847, 0, 54.47102126354997],
angles=[0, 14.4423397293289, 0, 13.1106455942161],
ge_avgs=[array([0., 0., 0., 0.]), array([ 48.63126676,  -6.49062513,  85.46995301, -15.97821486]), array([0., 0., 0., 0.]), array([34.19298312, 50.05795509, 93.8648478 , 36.16018922])],
counts_calib=[[7847, 533, 1488, 132], [1360, 6702, 265, 1673], [1919, 115, 7446, 520], [449, 1672, 1688, 6191], [1002, 7058, 130, 1810], [314, 1700, 955, 7031]]


  0%|          | 0/1 [00:00<?, ?it/s]

plotting shots


  0%|          | 0/6 [00:00<?, ?it/s]

Qubit (1) ge
ge fidelity (%): 79.71999999999998
Qubit (3) gf
gf fidelity (%): 27.514963496349655
thresholds_f=[0, 60.243054009284236, 0, -78.29633553673574],
angles_f=[0, 13.13607665360751, 0, 72.68401726725773],
gf_avgs=[array([0., 0., 0., 0.]), array([ 37.26142827,  -4.75169928,  85.20805843, -15.94105907]), array([0., 0., 0., 0.]), array([-2.38184219, 97.28566855,  2.6155858 , 81.25652441])],
counts_calib_f=[[7164, 2239, 463, 134], [7147, 1979, 707, 167], [1060, 385, 6293, 2262], [1162, 362, 6630, 1846], [4375, 4709, 502, 414], [873, 787, 4427, 3913]]


  0%|          | 0/1 [00:00<?, ?it/s]

plotting shots


In [131]:
# # TESTING
# rbprog = meas.SimultaneousRBEgGfExperiment(config_file=config_path)
# temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240617\\00011_rb_EgGf_qubit31.h5')
# rbprog.data = temp_data
# rbprog.cfg = AttrDict(attrs['config'])
# rbprog.calib_order = attrs['calib_order']

rbprog.analyze(fit=True)
rbprog.display(fit=True, show_all_vars=False)

[1.]
counts calib total shape (1, 6, 8)
[[[7847  533 1488  132 7164 2239  463  134]
  [1360 6702  265 1673 7147 1979  707  167]
  [1919  115 7446  520 1060  385 6293 2262]
  [ 449 1672 1688 6191 1162  362 6630 1846]
  [1002 7058  130 1810 4375 4709  502  414]
  [ 314 1700  955 7031  873  787 4427 3913]]]
counts raw shape (1, 1, 1, 8)


  0%|          | 0/1 [00:00<?, ?it/s]

minimization error 0.09695383358561817
counts raw [0.1013 0.794  0.0542 0.0505 0.3465 0.4659 0.1082 0.0794]
counts_corrected [[-1.35676314e-16  7.49567945e-17  3.43994604e-02 -1.45117238e-16
   9.65600540e-01 -3.05389861e-17]]
poplns_2q_loops shape (1, 1, 1, 6)
poplns_2q shape (1, 1, 6)
shape sum prob_eg + prob_gf (1, 1)
shape average sum over each depth (1,) should equal (1,)
Attempted to init fitparam 1 to 0.0, which is out of bounds 0 to 9.000000000000002. Instead init to 4.500000000000001
Attempted to init fitparam 2 to 1.0000000000000002, which is out of bounds 0 to 1.0000000000000002. Instead init to 0.5000000000000001
fit1 p1, a, offset [0.17060099 3.77060099 0.35673174]
fit2 a0, b0, c0, p2 [5.27893477e-11 1.76854189e-01 1.03045128e-02 4.10304508e-01]
Attempted to init fitparam 1 to 0.0, which is out of bounds 0 to 0.3095951434395348. Instead init to 0.1547975717197674
Attempted to init fitparam 2 to 0.03439946038217053, which is out of bounds 0 to 0.03439946038217053. Instead i

 C:\Users\slab\AppData\Roaming\Python\Python39\site-packages\scipy\optimize\_minpack_py.py: 1010

In [ ]:
rbprog.save_data()

### RUN RB

In [112]:
qubits = [3, 1]
qDrive = 3

# qubits = [3, 1]
# qDrive = 3

full_mux_expt = False
resonator_reset = [0, 2, 3] if full_mux_expt else None

use_robust_pulses = True

#### RB

In [113]:
step_sb = 4
end_sb = 110

step_leak = 50
end_leak = 400
sb = np.arange(1, end_sb+step_sb, step_sb)
lk = np.arange(sb[-1]+step_leak, end_leak+step_leak, step_leak)
depths = np.concatenate((sb, lk))
print('depths', len(depths))
print(depths)

plt.figure()
plt.plot(depths, np.ones_like(depths), 'o')
plt.show()

depths 35
[  1   5   9  13  17  21  25  29  33  37  41  45  49  53  57  61  65  69
  73  77  81  85  89  93  97 101 105 109 113 163 213 263 313 363 413]


In [114]:
rbprog = meas.SimultaneousRBEgGfExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"rb_EgGf_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

gate_char = None
# gate_char = 'X'

rbprog.cfg.expt = dict(
    depths=depths,
    variations=20,
    reps=3000, # number averages per unique sequence need at least 10,000 for a good amount of error per point!!
    loops=5,
    gate_char=gate_char, # single qubit clifford gate (str) to characterize. if not None, runs interleaved RB instead of regular RB
    qubits=qubits,
    qDrive=qDrive,
    singleshot_reps=20000, # reps per state for singleshot calibration
    post_process='threshold', # 'threshold' (uses single shot binning), 'scale' (scale by ge_avgs), or None

    measure_f_only=False,
    measure_f=[qDrive],
    add_phase=False,

    # cool_qubits=[0, 1],

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    use_robust_pulses=use_robust_pulses,

    validate_variations=True,

)

print(rbprog.cfg)

import Pyro4.util
try:
    rbprog.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-389.975, 5795.761666663333, 502.368233333334, -228.9], 'gain': [0.5, 0.6, 0.4, 0.3], 'readout_length': [5, 20, 10, 8], 'frequency_ef': [-391.91408405, 5795.76166666, 500.44439227, -230.30732892], 'gain_ef': [0.05, 0.6, 0.4, 0.25], 'readout_length_ef': [1, 20, 10, 8], 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4111.144451095202, 4105.783958808882, 4111.0028964412795, 4110.5966963096635, 3442.9795552267146, 3448.3315918098933, 3447.233490336077, 3447.0285717924894, 4762.767009306428, 4761.793655204115, 4762.902435440665, 4762.367822139689, 4380.662337410765, 4379.890920412824, 4380.6668186549305, 4381.242475074487], 'f_ef': [3885.3117884920853, 0.0, 0.0, 0.0, 0.0, 3347.8480888529375, 0.0, 0.0, 4571.9056482340775, 4571.070566218902, 4572.9717022650675, 0.0, 4211.5805515224565, 4207.169953136575, 0.0, 4209.889752285018], 'f_EgGf': [2686.53594683, 0, 2017.3, 2414.92850999], 'f_EgGf_Q_half':

  0%|          | 0/35 [00:00<?, ?it/s]

gate_list_variations= [[['Y/2', 'Y/2'], ['I', 'I'], ['X/2,Y/2,-X/2', 'I'], ['Y/2,X/2', 'Y/2,X/2'], ['X/2,Y/2,X/2', 'X/2,Y/2,X/2'], ['-X/2,Y', '-X/2,Y'], ['X,Y', 'I'], ['-X/2,Y', '-X/2,Y'], ['X/2,-Y/2', 'X/2,-Y/2'], ['-X/2,Y', '-X/2,Y'], ['-Y/2', '-Y/2'], ['Y/2,-X/2', 'Y/2,-X/2'], ['X', 'X'], ['-X/2,-Y/2', '-X/2,-Y/2'], ['-X/2', '-X/2'], ['Y/2', 'Y/2'], ['X', 'X'], ['Y/2,-X/2', 'Y/2,-X/2'], ['Y/2,X/2', 'Y/2,X/2'], ['X,Y', 'I']], [['X/2,Y/2,X/2', 'Y/2,X', '-Y/2,-X/2', '-Y/2,X/2', '-Y/2,-X/2', 'X/2,-Y/2'], ['X/2,Y/2,X/2', 'I', 'X/2,Y', 'X/2,Y/2', '-X/2,Y/2', 'X/2,-Y/2'], ['X/2,Y', 'X/2,Y/2,X/2', 'Y/2,X', 'Y/2', '-Y/2,X/2', 'Y'], ['X/2,-Y/2,-X/2', 'X/2,Y/2,X/2', '-Y/2,X', 'Y/2', '-Y/2', 'Y/2,X'], ['Y/2,X/2', 'X/2', 'X/2,Y/2,X/2', 'X,Y', 'Y/2,X', '-Y/2'], ['-X/2,Y/2,-X/2', 'Y/2,-X/2', '-Y/2', '-Y/2,-X/2', 'Y/2', 'I'], ['-Y/2', 'I', 'X/2,-Y/2,-X/2', '-X/2,-Y/2', 'X/2,Y/2,-X/2', '-X/2,Y/2,-X/2'], ['X/2,Y/2,X/2', 'X,Y', 'X', 'Y/2', '-X/2', 'Y/2'], ['-X/2,-Y/2', '-X/2,Y', 'X/2,Y', 'X/2', '-Y/2,

  0%|          | 0/5 [00:00<?, ?it/s]

Beginning loop 0


...

Qubit (1) ge
ge fidelity (%): 79.17999999999999 	 angle (deg): 16.003510611728082 	 threshold ge: 60.845058315108076
Qubit (3) ge
ge fidelity (%): 77.70000000000002 	 angle (deg): -125.37467346003761 	 threshold ge: 51.76869534540279
thresholds=[0, 60.845058315108076, 0, 51.76869534540279],
angles=[0, 16.003510611728082, 0, -125.37467346003761],
ge_avgs=[array([0., 0., 0., 0.]), array([ 36.82612507,  -7.00835805,  83.49712375, -20.39414636]), array([0., 0., 0., 0.]), array([-57.28659734, -14.99362801, -92.71388613,  34.90410645])],
counts_calib=[[17212, 1351, 1297, 140], [2813, 15367, 170, 1650], [2546, 195, 16105, 1154], [427, 2313, 2597, 14663], [1906, 16000, 155, 1939], [303, 2083, 1875, 15739]]


  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Qubit (1) ge
ge fidelity (%): 80.235
Qubit (3) gf
gf fidelity (%): 33.192912645632255
thresholds_f=[0, 59.358817468600364, 0, -77.77910714825293],
angles_f=[0, 14.101175787701846, 0, -7.60865204431857],
gf_avgs=[array([0., 0., 0., 0.]), array([ 35.23034883,  -8.73156971,  82.70223472, -20.65671892]), array([0., 0., 0., 0.]), array([-93.90245669,  15.85486338, -73.66135555,  18.55871391])],
counts_calib_f=[[13874, 5016, 803, 307], [13859, 4384, 1380, 377], [2012, 840, 12449, 4699], [2002, 585, 13404, 4009], [7306, 11010, 746, 938], [1231, 1347, 7715, 9707]]


  0%|          | 0/35 [00:00<?, ?it/s]

In [115]:
rbprog.save_data()

Saving S:\QRAM\qram_4QR2\data\data_241025\00009_rb_EgGf_qubit31.h5


'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00009_rb_EgGf_qubit31.h5'

In [116]:
# # TESTING
# rbprog = meas.SimultaneousRBEgGfExperiment(config_file=config_path)
# temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240617\\00011_rb_EgGf_qubit31.h5')
# rbprog.data = temp_daa
# rbprog.cfg = AttrDict(attrs['config'])
# rbprog.calib_order = attrs['calib_order']

rbprog.analyze(fit=True)
rbprog.display(fit=True, show_all_vars=False)

[  1.   5.   9.  13.  17.  21.  25.  29.  33.  37.  41.  45.  49.  53.
  57.  61.  65.  69.  73.  77.  81.  85.  89.  93.  97. 101. 105. 109.
 113. 163. 213. 263. 313. 363. 413.]
counts calib total shape (5, 6, 8)
[[[17421  1405  1049   125 13808  5203   721   268]
  [ 2939 15316   205  1540 13972  4507  1169   352]
  [ 2483   190 16130  1197  1971   663 13110  4256]
  [  435  2196  2461 14908  1880   647 13277  4196]
  [ 2054 15350   213  2383  7188 11312   667   833]
  [  388  2059  2064 15489  1200  1519  7308  9973]]

 [[17067  1408  1360   165 14590  4321   840   249]
  [ 2775 14636   287  2302 14672  3635  1375   318]
  [ 2456   235 15995  1314  2003   567 13822  3608]
  [  450  2164  2577 14809  2246   643 13704  3407]
  [ 1941 13925   239  3895  8369 10069   753   809]
  [  402  1951  2178 15469  1363  1360  8299  8978]]

 [[17210  1345  1320   125 14751  4142   879   228]
  [ 2593 15314   244  1849 14677  3576  1416   331]
  [ 2473   217 16095  1215  2034   620 13500  3846]
  

  0%|          | 0/35 [00:00<?, ?it/s]

minimization error 0.019552400948060412
counts raw [0.11533333 0.33633333 0.47033333 0.078      0.199      0.252
 0.37533333 0.17366667]
counts_corrected [[-9.51880207e-18 -3.41856417e-18  5.69654240e-01 -2.16220126e-18
   4.20233150e-01  1.01126100e-02]]
minimization error 0.01571732943105929
counts raw [0.11633333 0.00733333 0.831      0.04533333 0.09066667 0.03466667
 0.66233333 0.21233333]
counts_corrected [[ 4.28029726e-18  1.06948544e-17  1.00000000e+00 -1.28014045e-16
   9.97537710e-18  4.65253428e-18]]
minimization error 0.009369888949063687
counts raw [0.137      0.25233333 0.536      0.07466667 0.19666667 0.211
 0.427      0.16533333]
counts_corrected [[2.73003013e-02 2.33172315e-18 6.51095607e-01 5.15600097e-26
  3.19638632e-01 1.96546020e-03]]
minimization error 0.02482021606092811
counts raw [0.09466667 0.714      0.072      0.11933333 0.298      0.545
 0.08266667 0.07433333]
counts_corrected [[-1.32066855e-17 -1.57978220e-18  6.84703502e-02  2.45417258e-18
   9.16968054e-

  0%|          | 0/35 [00:00<?, ?it/s]

minimization error 0.03110785163187889
counts raw [0.134      0.275      0.53266667 0.05833333 0.248      0.184
 0.46833333 0.09966667]
counts_corrected [[ 9.74193885e-03  9.49069673e-02  6.39297844e-01 -2.66980480e-17
   2.56053250e-01  1.02616860e-17]]
minimization error 0.0427535671026406
counts raw [0.118      0.00233333 0.84833333 0.03133333 0.096      0.02233333
 0.73833333 0.14333333]
counts_corrected [[ 4.44527431e-17  2.45800902e-17  1.00000000e+00  1.68728861e-17
  -1.18221441e-16 -4.68940032e-18]]
minimization error 0.019753697463463078
counts raw [0.16933333 0.21366667 0.553      0.064      0.251      0.16166667
 0.47433333 0.113     ]
counts_corrected [[ 6.52336214e-02  6.46317529e-02  6.63086854e-01 -1.67749431e-17
   2.07047772e-01 -2.64642703e-17]]
minimization error 0.02775216268652008
counts raw [0.16       0.62066667 0.08666667 0.13266667 0.397      0.41966667
 0.11633333 0.067     ]
counts_corrected [[ 5.22596107e-02  4.62122732e-02  1.01283876e-01 -5.00364956e-17
 

...

minimization error 0.018148683291379482
counts raw [0.10466667 0.265      0.49933333 0.131      0.20566667 0.221
 0.44166667 0.13166667]
counts_corrected [[ 2.59444497e-17 -5.65224677e-25  5.97917878e-01  3.09492736e-02
   3.47518288e-01  2.36145611e-02]]
minimization error 0.03532843912644754
counts raw [0.11933333 0.00866667 0.82       0.052      0.09633333 0.023
 0.729      0.15166667]
counts_corrected [[-9.66632963e-17 -5.79516169e-18  1.00000000e+00 -1.30036091e-16
  -2.41205660e-18 -1.12218334e-17]]
minimization error 0.008779922484791808
counts raw [0.14833333 0.25633333 0.50933333 0.086      0.22566667 0.18166667
 0.46433333 0.12833333]
counts_corrected [[ 3.91189351e-02  1.13261120e-02  6.28525409e-01  2.00625344e-02
   3.00967009e-01 -7.36391270e-19]]
minimization error 0.033961365943974677
counts raw [0.102      0.7        0.07733333 0.12066667 0.328      0.489
 0.10366667 0.07933333]
counts_corrected [[-3.46860487e-17 -1.15650332e-15  8.34630182e-02 -2.48271610e-18
   8.973

  0%|          | 0/35 [00:00<?, ?it/s]

minimization error 0.023389017843422524
counts raw [0.125      0.248      0.53733333 0.08966667 0.225      0.20833333
 0.45433333 0.11233333]
counts_corrected [[ 1.50900768e-02  2.12733307e-18  6.57782512e-01  1.49036451e-02
   3.12223766e-01 -8.34863196e-18]]
minimization error 0.04045647180750072
counts raw [0.12333333 0.00333333 0.847      0.02633333 0.09566667 0.03366667
 0.69933333 0.17133333]
counts_corrected [[ 4.05033380e-17  5.82973740e-18  1.00000000e+00 -2.16403947e-17
  -3.71913757e-17 -1.42082654e-16]]
minimization error 0.02428550132067053
counts raw [0.17866667 0.22466667 0.54766667 0.049      0.243      0.16266667
 0.49333333 0.101     ]
counts_corrected [[ 5.59386739e-02  6.00240936e-02  6.85323832e-01 -9.16464676e-18
   1.98713400e-01  1.30230296e-17]]
minimization error 0.0034242084665185124
counts raw [0.15133333 0.673      0.08333333 0.09233333 0.40466667 0.43166667
 0.10566667 0.058     ]
counts_corrected [[0.05456424 0.06738498 0.08974647 0.0067276  0.75956983 0.

  0%|          | 0/35 [00:00<?, ?it/s]

minimization error 0.0132653889062636
counts raw [0.11366667 0.32733333 0.48133333 0.07766667 0.18533333 0.23466667
 0.40266667 0.17733333]
counts_corrected [[-2.62307015e-17  1.25078557e-17  5.98307884e-01  2.17444765e-18
   3.84406227e-01  1.72858896e-02]]
minimization error 0.049309347627785535
counts raw [0.12233333 0.01033333 0.80366667 0.06366667 0.08266667 0.02066667
 0.70633333 0.19033333]
counts_corrected [[-3.85687238e-18  4.29654889e-17  9.87731928e-01  1.22680725e-02
  -1.02631192e-17 -3.88229291e-17]]
minimization error 0.01806532137266423
counts raw [0.141      0.235      0.55066667 0.07333333 0.21266667 0.17633333
 0.46566667 0.14533333]
counts_corrected [[ 2.62650560e-02  6.13696294e-02  6.77592267e-01  1.55554819e-02
   2.19217566e-01 -1.33360822e-18]]
minimization error 0.019796737234691424
counts raw [0.10266667 0.71233333 0.07833333 0.10666667 0.30433333 0.51133333
 0.093      0.09133333]
counts_corrected [[ 9.40690373e-19 -8.37011929e-17  8.47413737e-02  1.28402866

#### IRB

In [117]:
step_sb = 2
end_sb = 70

step_leak = 25
end_leak = 200
sb = np.arange(1, end_sb+step_sb, step_sb)
lk = np.arange(sb[-1]+step_leak, end_leak+step_leak, step_leak)
depths = np.concatenate((sb, lk))
print('depths', len(depths))
print(depths)

plt.figure()
plt.plot(depths, np.ones_like(depths), 'o')
plt.show()

depths 42
[  1   3   5   7   9  11  13  15  17  19  21  23  25  27  29  31  33  35
  37  39  41  43  45  47  49  51  53  55  57  59  61  63  65  67  69  71
  96 121 146 171 196 221]


In [118]:
rbprog = meas.SimultaneousRBEgGfExperiment(
    soccfg=soc,
    path=expt_path,
    prefix=f"rb_EgGf_qubit{qubits[0]}{qubits[1]}",
    config_file=config_path,
)

# gate_char = None
gate_char = 'X'

rbprog.cfg.expt = dict(
    depths=depths,
    variations=20,
    reps=3000, # number averages per unique sequence need at least 10,000 for a good amount of error per point!!
    loops=5,
    gate_char=gate_char, # single qubit clifford gate (str) to characterize. if not None, runs interleaved RB instead of regular RB
    qubits=qubits,
    qDrive=qDrive,
    singleshot_reps=20000, # reps per state for singleshot calibration
    post_process='threshold', # 'threshold' (uses single shot binning), 'scale' (scale by ge_avgs), or None

    measure_f_only=False,
    measure_f=[qDrive],
    add_phase=False,

    # cool_qubits=[0, 1],

    full_mux_expt=full_mux_expt,
    resonator_reset=resonator_reset,
    use_robust_pulses=use_robust_pulses,

    validate_variations=True,

)

print(rbprog.cfg)

import Pyro4.util
try:
    rbprog.acquire(progress=True, debug=False)
except Exception:
    print("Pyro traceback:")
    print("".join(Pyro4.util.getPyroTraceback()))

{'device': {'readout': {'trig_offset': 150, 'lo_sideband': [1, 1, 1, 1], 'frequency': [-389.975, 5795.761666663333, 502.368233333334, -228.9], 'gain': [0.5, 0.6, 0.4, 0.3], 'readout_length': [5, 20, 10, 8], 'frequency_ef': [-391.91408405, 5795.76166666, 500.44439227, -230.30732892], 'gain_ef': [0.05, 0.6, 0.4, 0.25], 'readout_length_ef': [1, 20, 10, 8], 'relax_delay': [450, 450, 450, 450]}, 'qubit': {'f_ge': [4111.144451095202, 4105.783958808882, 4111.0028964412795, 4110.5966963096635, 3442.9795552267146, 3448.3315918098933, 3447.233490336077, 3447.0285717924894, 4762.767009306428, 4761.793655204115, 4762.902435440665, 4762.367822139689, 4380.662337410765, 4379.890920412824, 4380.6668186549305, 4381.242475074487], 'f_ef': [3885.3117884920853, 0.0, 0.0, 0.0, 0.0, 3347.8480888529375, 0.0, 0.0, 4571.9056482340775, 4571.070566218902, 4572.9717022650675, 0.0, 4211.5805515224565, 4207.169953136575, 0.0, 4209.889752285018], 'f_EgGf': [2686.53594683, 0, 2017.3, 2414.92850999], 'f_EgGf_Q_half':

  0%|          | 0/42 [00:00<?, ?it/s]

gate_list_variations= [[['-X/2,Y/2,-X/2', 'X', 'I'], ['X/2,-Y/2,-X/2', 'X', 'X/2,Y/2,X/2'], ['-X/2', 'X', 'X/2'], ['Y/2,-X/2', 'X', '-Y/2,X/2'], ['-Y/2,X', 'X', 'Y/2'], ['-X/2,Y/2', 'X', 'X/2,Y/2'], ['X/2,Y/2,X/2', 'X', 'I'], ['-Y/2,X/2', 'X', 'Y/2,-X/2'], ['-X/2,Y/2,-X/2', 'X', 'I'], ['X/2,-Y/2', 'X', '-X/2,-Y/2'], ['-Y/2,-X/2', 'X', 'Y/2,X/2'], ['X', 'X', 'I'], ['X/2,-Y/2,-X/2', 'X', 'X/2,Y/2,X/2'], ['I', 'X', 'X'], ['-X/2,Y/2,-X/2', 'X', 'I'], ['-X/2,Y/2,-X/2', 'X', 'I'], ['X,Y', 'X', 'Y'], ['-Y/2,X/2', 'X', 'Y/2,-X/2'], ['Y/2,X/2', 'X', '-Y/2,-X/2'], ['Y/2,X/2', 'X', '-Y/2,-X/2']], [['X/2,-Y/2,-X/2', 'X', 'X/2,Y/2', 'X', '-X/2,-Y/2', 'X', '-X/2'], ['-Y/2,X/2', 'X', 'Y', 'X', '-Y/2,-X/2', 'X', '-X/2,-Y/2'], ['Y/2,-X/2', 'X', '-X/2', 'X', 'Y/2,X', 'X', '-X/2,Y/2'], ['-X/2,Y', 'X', '-X/2,-Y/2', 'X', '-X/2,Y/2,-X/2', 'X', 'I'], ['I', 'X', '-Y/2,X/2', 'X', 'X/2,Y/2', 'X', 'I'], ['Y/2,-X/2', 'X', 'X,Y', 'X', '-Y/2,X/2', 'X', '-X/2,Y/2'], ['-X/2,Y/2,-X/2', 'X', 'X/2,Y/2,-X/2', 'X', 'X/2,Y

  0%|          | 0/5 [00:00<?, ?it/s]

Beginning loop 0


...

Qubit (1) ge
ge fidelity (%): 79.13971773588679 	 angle (deg): 16.496673118603418 	 threshold ge: 60.95041646692185
Qubit (3) ge
ge fidelity (%): 78.83969948497426 	 angle (deg): 21.618833366078597 	 threshold ge: 54.17664540926378
thresholds=[0, 60.95041646692185, 0, 54.17664540926378],
angles=[0, 16.496673118603418, 0, 21.618833366078597],
ge_avgs=[array([0., 0., 0., 0.]), array([ 36.69983005,  -7.79876676,  82.77480199, -21.44388519]), array([0., 0., 0., 0.]), array([40.84693271, 43.68745968, 98.5870593 , 20.80456886])],
counts_calib=[[17573, 1223, 1120, 84], [2702, 15262, 227, 1809], [2794, 179, 15977, 1050], [457, 2174, 2694, 14675], [2308, 15827, 176, 1689], [375, 2010, 2173, 15442]]


  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Qubit (1) ge
ge fidelity (%): 80.71000000000001
Qubit (3) gf
gf fidelity (%): 35.83999999999998
thresholds_f=[0, 60.99736983781928, 0, -73.60717171643456],
angles_f=[0, 15.927515561804732, 0, 69.85545364091514],
gf_avgs=[array([0., 0., 0., 0.]), array([ 35.56415583,  -7.81800388,  83.52188718, -21.50402634]), array([0., 0., 0., 0.]), array([-2.13497014, 94.40976839,  5.48786644, 73.62946418])],
counts_calib_f=[[14640, 4341, 788, 231], [14465, 4037, 1205, 293], [2215, 625, 13475, 3685], [2146, 610, 13855, 3389], [7631, 10938, 651, 780], [1337, 1282, 8167, 9214]]


  0%|          | 0/42 [00:00<?, ?it/s]

In [119]:
rbprog.save_data()

Saving S:\QRAM\qram_4QR2\data\data_241025\00010_rb_EgGf_qubit31.h5


'S:\\QRAM\\qram_4QR2\\data\\data_241025\\00010_rb_EgGf_qubit31.h5'

In [120]:
# # TESTING
# rbprog = meas.SimultaneousRBEgGfExperiment(config_file=config_path)
# temp_data, attrs = prev_data('s:\Connie\experiments\qramLL_4QR2\data\data_240617\\00011_rb_EgGf_qubit31.h5')
# rbprog.data = temp_data
# rbprog.cfg = AttrDict(attrs['config'])
# rbprog.calib_order = attrs['calib_order']

rbprog.analyze(fit=True)
rbprog.display(fit=True, show_all_vars=False)

[  1.   3.   5.   7.   9.  11.  13.  15.  17.  19.  21.  23.  25.  27.
  29.  31.  33.  35.  37.  39.  41.  43.  45.  47.  49.  51.  53.  55.
  57.  59.  61.  63.  65.  67.  69.  71.  96. 121. 146. 171. 196. 221.]
counts calib total shape (5, 6, 8)
[[[17311  1378  1183   128 14724  4182   878   216]
  [ 2849 14945   271  1935 14770  3659  1310   261]
  [ 2624   197 15852  1327  2331   615 13677  3377]
  [  512  2106  2749 14633  2013   481 14349  3157]
  [ 2212 14605   204  2979  8073 10256   821   850]
  [  386  1935  2153 15526  1308  1302  8359  9031]]

 [[17468  1387  1023   122 13613  5281   842   264]
  [ 3549 14349   314  1788 13579  4881  1149   391]
  [ 2502   200 16011  1287  1872   787 12324  5017]
  [  600  2301  3471 13628  2130   658 13073  4139]
  [ 2029 15122   222  2627  7387 11001   707   905]
  [  346  1997  1967 15690  1261  1441  7743  9555]]

 [[17085  1470  1297   148 14744  4283   759   214]
  [ 2557 14778   293  2372 14398  3985  1301   316]
  [ 2534   217 1588

  0%|          | 0/42 [00:00<?, ?it/s]

minimization error 0.001683121285610009
counts raw [0.16533333 0.08433333 0.677      0.07333333 0.185      0.072
 0.59066667 0.15233333]
counts_corrected [[4.73907309e-02 3.88672335e-02 8.48917075e-01 1.43823098e-17
  5.73085841e-02 7.51637709e-03]]
minimization error 0.01484454042381518
counts raw [0.13266667 0.31433333 0.45866667 0.09433333 0.23166667 0.21966667
 0.40833333 0.14033333]
counts_corrected [[6.55608178e-03 6.83689249e-03 5.73404579e-01 1.78024071e-18
  4.03148915e-01 1.00535319e-02]]
minimization error 0.019443645393350154
counts raw [0.19633333 0.218      0.50833333 0.07733333 0.23       0.15666667
 0.48766667 0.12566667]
counts_corrected [[ 7.82241518e-02  1.36495180e-02  6.47411221e-01  1.55318394e-02
   2.45183269e-01 -9.46698350e-18]]
minimization error 0.017777170504924694
counts raw [0.23333333 0.20766667 0.482      0.077      0.255      0.17133333
 0.42866667 0.145     ]
counts_corrected [[ 1.34378083e-01 -1.28341959e-17  5.97338216e-01 -1.30854346e-17
   2.50149

  0%|          | 0/42 [00:00<?, ?it/s]

minimization error 0.061574050786424285
counts raw [0.18166667 0.065      0.719      0.03433333 0.15233333 0.07266667
 0.636      0.139     ]
counts_corrected [[ 5.19510141e-02  4.63468904e-02  9.01702096e-01 -2.59722936e-18
   1.24502447e-17 -3.87992043e-17]]
minimization error 0.020743826768884824
counts raw [0.145      0.27133333 0.504      0.07966667 0.227      0.23166667
 0.40666667 0.13466667]
counts_corrected [[ 3.80738239e-02  2.24705059e-02  6.12529596e-01  9.63944382e-18
   3.26926075e-01 -9.37943349e-18]]
minimization error 0.026061752539518063
counts raw [0.18333333 0.18633333 0.56466667 0.06566667 0.23033333 0.16466667
 0.46466667 0.14033333]
counts_corrected [[ 7.67803181e-02  7.19443156e-02  6.92088271e-01 -7.24392651e-18
   1.59187096e-01 -3.36507519e-17]]
minimization error 0.03601203416510819
counts raw [0.17933333 0.20733333 0.557      0.05633333 0.24733333 0.17633333
 0.45633333 0.12      ]
counts_corrected [[ 6.99487323e-02  9.98477357e-02  6.71162831e-01  7.977648

...

minimization error 0.010789555662909386
counts raw [0.148      0.07033333 0.71433333 0.06733333 0.16933333 0.06633333
 0.602      0.16233333]
counts_corrected [[ 3.20515826e-02  6.09129673e-02  8.87974151e-01 -4.90743564e-18
   1.90612994e-02 -3.75868178e-17]]
minimization error 0.008158536407869982
counts raw [0.12766667 0.29966667 0.45466667 0.118      0.231      0.21833333
 0.411      0.13966667]
counts_corrected [[1.27899829e-02 4.05507264e-02 5.70541691e-01 1.50942658e-02
  3.61023334e-01 1.64487365e-19]]
minimization error 0.006530851500633
counts raw [0.159      0.214      0.53933333 0.08766667 0.22666667 0.17366667
 0.46       0.13966667]
counts_corrected [[ 5.29437349e-02  3.62118533e-02  6.67038188e-01 -9.63415196e-17
   2.43806224e-01 -1.99768233e-16]]
minimization error 0.001925844671732855
counts raw [0.13733333 0.239      0.515      0.10866667 0.217      0.19033333
 0.446      0.14666667]
counts_corrected [[0.02823184 0.02279494 0.64191439 0.00465192 0.29890012 0.00350679

  0%|          | 0/42 [00:00<?, ?it/s]

minimization error 0.02048587550690432
counts raw [0.18966667 0.06633333 0.69466667 0.04933333 0.18366667 0.071
 0.59666667 0.14866667]
counts_corrected [[ 8.35052496e-02  2.87180771e-02  8.63384772e-01 -7.00603220e-17
   2.43919011e-02 -7.56753979e-18]]
minimization error 0.005352246938651372
counts raw [0.17833333 0.27666667 0.47       0.075      0.254      0.216
 0.407      0.123     ]
counts_corrected [[8.67137123e-02 2.17778840e-17 5.82052782e-01 2.45960068e-03
  3.28773905e-01 8.81795429e-18]]
minimization error 0.031203110499097544
counts raw [0.23733333 0.182      0.52533333 0.05533333 0.256      0.14866667
 0.49333333 0.102     ]
counts_corrected [[ 1.45629482e-01  2.51458473e-02  6.62101901e-01  2.96426836e-03
   1.64158502e-01 -3.70082282e-17]]
minimization error 0.02199221004120274
counts raw [0.229      0.194      0.52366667 0.05333333 0.25966667 0.15566667
 0.472      0.11266667]
counts_corrected [[ 1.38528044e-01  2.77751395e-02  6.52860041e-01  7.46321242e-18
   1.80836

  0%|          | 0/42 [00:00<?, ?it/s]

minimization error 0.015498573724917065
counts raw [0.19366667 0.09033333 0.65533333 0.06066667 0.20266667 0.103
 0.52533333 0.169     ]
counts_corrected [[ 8.85120360e-02  8.52561839e-03  7.94066985e-01 -4.39605065e-29
   9.66828399e-02  1.22125206e-02]]
minimization error 0.003724996543302338
counts raw [0.13966667 0.313      0.45466667 0.09266667 0.228      0.23933333
 0.408      0.12466667]
counts_corrected [[2.05493706e-02 2.30127560e-17 5.54668682e-01 2.62185237e-02
  3.86686716e-01 1.18767073e-02]]
minimization error 0.014722608623410604
counts raw [0.19066667 0.23133333 0.51       0.068      0.20533333 0.18133333
 0.461      0.15233333]
counts_corrected [[ 6.45538996e-02 -5.78504138e-17  6.39105779e-01 -6.22562277e-18
   2.68616214e-01  2.77241070e-02]]
minimization error 0.002737690548787245
counts raw [0.19433333 0.23666667 0.50933333 0.05966667 0.251      0.17733333
 0.43833333 0.13333333]
counts_corrected [[8.15354660e-02 4.12805522e-02 6.28318420e-01 6.31289021e-28
  2.433